In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv("processed_USA_less.csv")
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),City,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,datetime_start_parsed,datetime_end_parsed,datetime_weathertime_parsed
0,3,39.865147,-84.058723,39.865147,-84.058723,0.01,Dayton,OH,62.0,62.0,91,29.68,10.0,7.0,0.02,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 05:46:00,2016-02-08 11:00:00,2016-02-08 05:58:00
1,2,39.928059,-82.831184,39.928059,-82.831184,0.01,Reynoldsburg,OH,62.0,62.0,100,29.65,10.0,7.0,0.00,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 06:07:59,2016-02-08 06:37:59,2016-02-08 05:51:00
2,2,39.063148,-84.032608,39.063148,-84.032608,0.01,Williamsburg,OH,36.0,33.3,100,29.67,10.0,3.5,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 06:49:27,2016-02-08 07:19:27,2016-02-08 06:56:00
3,3,39.747753,-84.205582,39.747753,-84.205582,0.01,Dayton,OH,35.1,31.0,96,29.64,9.0,4.6,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 07:23:34,2016-02-08 07:53:34,2016-02-08 07:38:00
4,2,39.627781,-84.188354,39.627781,-84.188354,0.01,Dayton,OH,36.0,33.3,89,29.65,6.0,3.5,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 07:39:07,2016-02-08 08:09:07,2016-02-08 07:53:00


In [3]:
# List of columns to convert to datetime
columns_to_convert = ['datetime_start_parsed','datetime_end_parsed', 'datetime_weathertime_parsed']

# Convert specified columns to datetime
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_datetime, errors='coerce')


In [4]:
columns_to_drop = ["City","State"]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [5]:
df=df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7700098 entries, 0 to 7700097
Data columns (total 30 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   Severity                     Int64         
 1   Start_Lat                    Float64       
 2   Start_Lng                    Float64       
 3   End_Lat                      Float64       
 4   End_Lng                      Float64       
 5   Distance(mi)                 Float64       
 6   Temperature(F)               Float64       
 7   Wind_Chill(F)                Float64       
 8   Humidity(%)                  Int64         
 9   Pressure(in)                 Float64       
 10  Visibility(mi)               Float64       
 11  Wind_Speed(mph)              Float64       
 12  Precipitation(in)            Float64       
 13  Weather_Condition            string        
 14  Amenity                      boolean       
 15  Bump                         boolean       
 16  

In [6]:
df1 = df.copy()

In [7]:
df1['Severity'].value_counts()

Severity
2    6135257
3    1295600
4     202047
1      67194
Name: count, dtype: Int64

In [8]:
severity_mapping = {
    1: 0,
    2: 0,
    3: 1,
    4: 1
}

df['Severity'] = df['Severity'].map(severity_mapping)

In [9]:
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,datetime_start_parsed,datetime_end_parsed,datetime_weathertime_parsed
0,1,39.865147,-84.058723,39.865147,-84.058723,0.01,62.0,62.0,91,29.68,10.0,7.0,0.02,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 05:46:00,2016-02-08 11:00:00,2016-02-08 05:58:00
1,0,39.928059,-82.831184,39.928059,-82.831184,0.01,62.0,62.0,100,29.65,10.0,7.0,0.0,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 06:07:59,2016-02-08 06:37:59,2016-02-08 05:51:00
2,0,39.063148,-84.032608,39.063148,-84.032608,0.01,36.0,33.3,100,29.67,10.0,3.5,0.0,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 06:49:27,2016-02-08 07:19:27,2016-02-08 06:56:00
3,1,39.747753,-84.205582,39.747753,-84.205582,0.01,35.1,31.0,96,29.64,9.0,4.6,0.0,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 07:23:34,2016-02-08 07:53:34,2016-02-08 07:38:00
4,0,39.627781,-84.188354,39.627781,-84.188354,0.01,36.0,33.3,89,29.65,6.0,3.5,0.0,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 07:39:07,2016-02-08 08:09:07,2016-02-08 07:53:00


In [10]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.utils import resample


# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df, columns=['Weather_Condition'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

# Separate classes
class_0 = df_encoded[df_encoded['Severity'] == 0]
class_1 = df_encoded[df_encoded['Severity'] == 1]

# Randomly select samples from class 0 to match the size of class 1
class_0_downsampled = resample(class_0, replace=False, n_samples=len(class_1), random_state=42)

# Concatenate the downsampled class 0 with class 1
balanced_data = pd.concat([class_0_downsampled, class_1])

# Split the data into features and target variable
X_balanced = balanced_data.drop('Severity', axis=1)
y_balanced = balanced_data['Severity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Define classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

# Precompute RFE results
rfe_results = {}
for clf_name, clf in classifiers.items():
    if not isinstance(clf, KNeighborsClassifier):
        rfe = RFE(estimator=clf, n_features_to_select=10)
        X_train_selected = rfe.fit_transform(X_train, y_train)
        X_test_selected = rfe.transform(X_test)
        rfe_results[clf_name] = {'selected_features': X.columns[rfe.support_],
                                 'X_train_selected': X_train_selected,
                                 'X_test_selected': X_test_selected}

# Iterate over classifiers
for clf_name, clf in classifiers.items():
    print(f"\nTraining {clf_name}:")

    # Skip KNeighborsClassifier for RFE
    if clf_name in rfe_results:
        # Extract precomputed RFE results
        selected_features = rfe_results[clf_name]['selected_features']
        X_train_selected = rfe_results[clf_name]['X_train_selected']
        X_test_selected = rfe_results[clf_name]['X_test_selected']

        # Train the model on the selected features
        clf.fit(X_train_selected, y_train)

        # Make predictions on the test set
        y_pred = clf.predict(X_test_selected)

        # Print selected features
        print(f"Selected Features: {selected_features}")

        # Print classification report, confusion matrix, and AUC
        print("Classification Report:")
        print(classification_report(y_test, y_pred))

        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        # Calculate AUC
        y_prob = clf.predict_proba(X_test_selected)[:, 1]
        auc = roc_auc_score(y_test, y_prob)
        print(f"AUC: {auc}")


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Randomly shuffle the rows of the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

# Split the shuffled dataset into 5 equal parts
df_parts = [df_shuffled.iloc[i:i + len(df_shuffled)//5] for i in range(0, len(df_shuffled), len(df_shuffled)//5)]

# Choose one part for analysis (for example, the first part)
df_selected = df_parts[0]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy for the selected part: {accuracy}")

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Model Accuracy for the selected part: 0.8688945598109116
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92    248531
           1       0.77      0.46      0.57     59473

    accuracy                           0.87    308004
   macro avg       0.83      0.71      0.75    308004
weighted avg       0.86      0.87      0.86    308004

Confusion Matrix:
[[240496   8035]
 [ 32346  27127]]


In [45]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train an XGBoost classifier
xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate the model
y_pred_xgb = xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"XGBoost Model Accuracy: {accuracy_xgb}")
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))


XGBoost Model Accuracy: 0.863258269373125
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92    248531
           1       0.73      0.46      0.56     59473

    accuracy                           0.86    308004
   macro avg       0.81      0.71      0.74    308004
weighted avg       0.85      0.86      0.85    308004

Confusion Matrix:
[[238663   9868]
 [ 32249  27224]]


In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, classification_report

# Build a simple neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predictions on the test set
nn_y_pred_proba = model.predict(X_test)
nn_y_pred = (nn_y_pred_proba > 0.5).astype("int32")

# Evaluate the model
nn_accuracy = accuracy_score(y_test, nn_y_pred)
print(f"Neural Network Model Accuracy: {nn_accuracy:.2f}")

# Additional evaluation metrics
print("Neural Network Classification Report:")
print(classification_report(y_test, nn_y_pred))


Epoch 1/10
38501/38501 [==============================] - 64s 2ms/step - loss: 0.4124 - accuracy: 0.8169 - val_loss: 0.3920 - val_accuracy: 0.8227
Epoch 2/10
38501/38501 [==============================] - 65s 2ms/step - loss: 0.3892 - accuracy: 0.8243 - val_loss: 0.3842 - val_accuracy: 0.8271
Epoch 3/10
38501/38501 [==============================] - 68s 2ms/step - loss: 0.3832 - accuracy: 0.8269 - val_loss: 0.3783 - val_accuracy: 0.8297
Epoch 4/10
38501/38501 [==============================] - 64s 2ms/step - loss: 0.3798 - accuracy: 0.8283 - val_loss: 0.3757 - val_accuracy: 0.8310
Epoch 5/10
38501/38501 [==============================] - 65s 2ms/step - loss: 0.3769 - accuracy: 0.8297 - val_loss: 0.3743 - val_accuracy: 0.8315
Epoch 6/10
38501/38501 [==============================] - 66s 2ms/step - loss: 0.3749 - accuracy: 0.8306 - val_loss: 0.3719 - val_accuracy: 0.8324
Epoch 7/10
38501/38501 [==============================] - 65s 2ms/step - loss: 0.3725 - accuracy: 0.8317 - val_loss: 0

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train a GradientBoostingClassifier
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Evaluate the Gradient Boosting model
accuracy_gb = gb_model.score(X_test, y_test)
print(f"Gradient Boosting Model Accuracy for the selected part: {accuracy_gb}")

# Generate predictions on the test set
y_pred_gb = gb_model.predict(X_test)

# Display classification report
print("Classification Report for Gradient Boosting:")
print(classification_report(y_test, y_pred_gb))

# Display confusion matrix
print("Confusion Matrix for Gradient Boosting:")
print(confusion_matrix(y_test, y_pred_gb))

Gradient Boosting Model Accuracy for the selected part: 0.8243074765262789
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.83      0.99      0.90    248531
           1       0.71      0.15      0.25     59473

    accuracy                           0.82    308004
   macro avg       0.77      0.57      0.58    308004
weighted avg       0.81      0.82      0.78    308004

Confusion Matrix for Gradient Boosting:
[[244805   3726]
 [ 50388   9085]]


In [52]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create individual models
model = RandomForestClassifier()
gb_model = GradientBoostingClassifier(random_state=42)
xgb_model = XGBClassifier(random_state=42)

# Create an ensemble model using VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('rf', model),
    ('gb', gb_model),
    ('xgb', xgb_model),
], voting='hard')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Predictions on the test set
ensemble_y_pred = ensemble_model.predict(X_test)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, ensemble_y_pred)
print(f"Ensemble Model Accuracy: {ensemble_accuracy:.2f}")

# Additional evaluation metrics
print("Ensemble Model Classification Report:")
print(classification_report(y_test, ensemble_y_pred))


Ensemble Model Accuracy: 0.86
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92    248531
           1       0.79      0.39      0.52     59473

    accuracy                           0.86    308004
   macro avg       0.83      0.68      0.72    308004
weighted avg       0.85      0.86      0.84    308004



In [ ]:
# Display confusion matrix
print("Confusion Matrix for Ensemble Voting Classifier:")
print(confusion_matrix(y_test, ensemble_y_pred))

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import randint

# Define the parameter distribution to sample from
param_dist = {
    'n_estimators': randint(100, 200),
    'max_depth': randint(10, 30),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
}

# Create a RandomForestClassifier
model = RandomForestClassifier(random_state=42)

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy', random_state=42
)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Get the best model
best_model_random = random_search.best_estimator_

# Predictions on the test set using the best model
y_pred_random_tuned = best_model_random.predict(X_test)

# Evaluate the tuned model
accuracy_random_tuned = accuracy_score(y_test, y_pred_random_tuned)
print(f"Tuned Model Accuracy: {accuracy_random_tuned:.2f}")

# Additional evaluation metrics for the tuned model
print("Classification Report (Tuned Model):")
print(classification_report(y_test, y_pred_random_tuned))

# Display confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_random_tuned))


Best Hyperparameters: {'bootstrap': False, 'max_depth': 27, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 161}
Tuned Model Accuracy: 0.86
Classification Report (Tuned Model):
              precision    recall  f1-score   support

           0       0.87      0.97      0.92    248531
           1       0.78      0.41      0.54     59473

    accuracy                           0.86    308004
   macro avg       0.83      0.69      0.73    308004
weighted avg       0.86      0.86      0.85    308004

Confusion Matrix for Random Forest Classifier:
[[241637   6894]
 [ 34827  24646]]


## Logistic

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Randomly shuffle the rows of the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

# Split the shuffled dataset into 5 equal parts
df_parts = [df_shuffled.iloc[i:i + len(df_shuffled)//5] for i in range(0, len(df_shuffled), len(df_shuffled)//5)]

# Choose one part for analysis (for example, the first part)
df_selected = df_parts[0]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a Logistic Regression model
logreg_model = LogisticRegression(random_state=42)

# Train the model
logreg_model.fit(X_train, y_train)

# Predictions on the test set
logreg_y_pred = logreg_model.predict(X_test)

# Evaluate the model
logreg_accuracy = accuracy_score(y_test, logreg_y_pred)
print(f"Logistic Regression Accuracy: {logreg_accuracy:.2f}")

# Additional evaluation metrics
print("Logistic Regression Classification Report:")
print(classification_report(y_test, logreg_y_pred))

# Display confusion matrix
print("Confusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, logreg_y_pred))

Logistic Regression Accuracy: 0.81
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      1.00      0.89    248531
           1       0.30      0.00      0.00     59473

    accuracy                           0.81    308004
   macro avg       0.55      0.50      0.45    308004
weighted avg       0.71      0.81      0.72    308004

Confusion Matrix for Logistic Regression:
[[248505     26]
 [ 59462     11]]


## KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Randomly shuffle the rows of the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

# Split the shuffled dataset into 5 equal parts
df_parts = [df_shuffled.iloc[i:i + len(df_shuffled)//5] for i in range(0, len(df_shuffled), len(df_shuffled)//5)]

# Choose one part for analysis (for example, the first part)
df_selected = df_parts[0]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build a K-Nearest Neighbors model
knn_model = KNeighborsClassifier()

# Train the model
knn_model.fit(X_train, y_train)

# Predictions on the test set
knn_y_pred = knn_model.predict(X_test)

# Evaluate the model
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print(f"KNN Accuracy: {knn_accuracy:.2f}")

# Additional evaluation metrics
print("KNN Classification Report:")
print(classification_report(y_test, knn_y_pred))

# Display confusion matrix
print("Confusion Matrix for KNN:")
print(confusion_matrix(y_test, knn_y_pred))


KNN Accuracy: 0.78
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.92      0.87    248531
           1       0.36      0.18      0.24     59473

    accuracy                           0.78    308004
   macro avg       0.59      0.55      0.56    308004
weighted avg       0.73      0.78      0.75    308004

Confusion Matrix for KNN:
[[229525  19006]
 [ 48820  10653]]


In [56]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [20],
    'min_samples_split': [5],
    # Add other hyperparameters you want to tune
}

# Create a RandomForestClassifier
model = RandomForestClassifier(random_state=42)

# Create GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_

# Predictions on the test set using the best model
y_pred_tuned = best_model.predict(X_test)

# Evaluate the tuned model
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned Model Accuracy: {accuracy_tuned:.2f}")

# Additional evaluation metrics for the tuned model
print("Classification Report (Tuned Model):")
print(classification_report(y_test, y_pred_tuned))

# Display confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_tuned))

KeyboardInterrupt: 

## Part 2

In [57]:
df_selected = df_parts[1]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy for the selected part: {accuracy}")

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


KeyboardInterrupt: 

## Part 3

In [ ]:

df_selected = df_parts[2]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy for the selected part: {accuracy}")

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


## Part 4

In [ ]:

df_selected = df_parts[3]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy for the selected part: {accuracy}")

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


## Part 5

In [ ]:

df_selected = df_parts[4]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy for the selected part: {accuracy}")

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
